In [40]:
import pickle
import pandas as pd

from numpy import array
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils.np_utils import to_categorical

In [77]:
images_folder = 'D:/datasets/flickr-30k-images'

curr_folder = "D:/YandexDisk/datasets/"
path_train = curr_folder + "captions-ru-train.csv"

path_features = curr_folder + "features.pkl"
path_sentences = curr_folder + "sentences-train.pkl"

# Загрузка данных

Мы собираемся обучить данные по всем фотографиям и подписям в обучающем наборе данных. Во время обучения мы будем отслеживать производительность модели в наборе данных разработки и использовать эту производительность, чтобы решить, когда сохранять модели в файл.

Модель, которую мы разработаем, будет генерировать подпись к фотографии, и подпись будет генерироваться по одному слову за раз.

Последовательность ранее сгенерированных слов будет предоставлена в качестве входных данных. Поэтому нам понадобится "первое слово", чтобы начать процесс генерации, и "последнее слово", чтобы сигнализировать об окончании подписи. Для этой цели мы будем использовать строки "startseq" и "endseq". Эти маркеры добавляются к загруженным описаниям по мере их загрузки. Важно сделать это сейчас, прежде чем мы закодируем текст, чтобы токены также были закодированы правильно.

# Множество изображений

In [65]:
def image_names_set(data):
    vals = set()

    for idx in data.index:
        vals.add(data.iat[idx, 0][:-4])

    return vals

def load_image_features(filename, data):
    all_features = pickle.load(open(filename, 'rb'))
    features = {k: all_features[k] for k in data}

    return features

In [79]:
train_df = pd.read_csv(path_train, delimiter='|')
train_features = load_image_features(path_features, image_names_set(train_df))
print('кол-во изображений ... %d' % len(train_features))

кол-во изображений ... 10902


# Закодировать знаки в числа

Текст описания необходимо будет закодировать в числа, прежде чем его можно будет представить модели в качестве входных данных или сравнить с предсказаниями модели.

Первым шагом в кодировании данных является создание согласованного сопоставления слов с уникальными целочисленными значениями. Keras предоставляет класс Tokenizer, который может изучить это сопоставление из загруженных данных описания.

Каждое описание будет разделено на слова. Модель будет предоставлена одним словом и фотографией и сгенерирует следующее слово. Затем первые два слова описания будут предоставлены модели в качестве входных данных вместе с изображением для создания следующего слова. Именно так будет обучаться модель.

In [18]:
with open (path_sentences, 'rb') as f:
    sentences = pickle.load(f)

In [23]:
def create_tokenizer(data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)

    return tokenizer

In [24]:
tokenizer = create_tokenizer(sentences)
vocab_size = len(tokenizer.word_index) + 1
print('размер словаря предложений ... %d' % vocab_size)

размер словаря предложений ... 26817


# Создание последовательности

Приведенная ниже функция с именем create_sequences(), учитывая токенизатор, максимальную длину последовательности и словарь всех описаний и фотографий, преобразует данные в пары ввода-вывода данных для обучения модели.

В модели есть два входных массива: один для признаков фотографии и один для закодированного текста. Существует один вывод для модели, который представляет собой закодированное следующее слово в текстовой последовательности.

Входной текст кодируется в виде целых чисел, которые будут передаваться на слой встраивания слов. Признаки изображения будут передаваться непосредственно в другую часть модели. Модель выдаст прогноз, который будет представлять собой распределение вероятностей по всем словам в словаре.

Таким образом, выходные данные будут представлять собой однократно закодированную версию каждого слова, представляющую идеализированное распределение вероятностей со значениями 0 во всех позициях слов, кроме фактической позиции слова, которая имеет значение 1.

In [ ]:
def max_length(data):
    return max(len(d.split()) for d in data)

def create_sequences(tokenizer, max_length, data, photos, vocab_size):
    X1, X2, y = list(), list(), list()

    for idx in data.index:
        seq = tokenizer.texts_to_sequences([data.iat[idx, 2]])[0]

        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]

            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

            X1.append(photos[data.iat[idx, 0][:-4]][0])
            X2.append(in_seq)
            y.append(out_seq)

    return array(X1), array(X2), array(y)